# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights


In [1]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

from __future__ import print_function

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [4]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
#     cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
#     X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    from keras.datasets import cifar10

    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    X_train, X_test = X_train.astype('float64'), X_test.astype('float64')
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000, 1)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000, 1)
Test data shape:  (1000, 3073)
Test labels shape:  (1000, 1)
dev data shape:  (500, 3073)
dev labels shape:  (500, 1)


## Softmax Classifier

Your code for this section will all be written inside **cs231n/classifiers/softmax.py**. 


In [40]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs231n/classifiers/softmax.py and implement the
# softmax_loss_naive function.

from cs231n.classifiers.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

loss: 2.423729
sanity check: 2.302585


## Inline Question 1:
Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

**Your answer:** *Fill this in*


In [70]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

print()
# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: 0.561505 analytic: 0.561505, relative error: 6.723229e-08
numerical: -0.811389 analytic: -0.811389, relative error: 4.437702e-10
numerical: -7.439153 analytic: -7.439153, relative error: 1.817988e-09
numerical: 2.486332 analytic: 2.486332, relative error: 4.000901e-09
numerical: -5.827712 analytic: -5.827712, relative error: 5.677363e-09
numerical: 0.948901 analytic: 0.948901, relative error: 2.070596e-08
numerical: 0.481770 analytic: 0.481770, relative error: 1.116953e-07
numerical: -3.448945 analytic: -3.448945, relative error: 1.231007e-08
numerical: 1.160203 analytic: 1.160203, relative error: 9.498643e-09
numerical: -3.609490 analytic: -3.609490, relative error: 9.860923e-09

numerical: 0.548718 analytic: 0.548718, relative error: 4.952961e-08
numerical: -0.056326 analytic: -0.056326, relative error: 9.629983e-07
numerical: 3.955880 analytic: 3.955880, relative error: 2.117430e-08
numerical: 0.970365 analytic: 0.970365, relative error: 6.323317e-08
numerical: 1.450335 a

In [77]:
X = np.array([[1,2,3,1],[2,3,4,1]])

y = np.array([[0],[2]])

w = np.array([[2,5,3],[4,6,3],[3,5,7],[4,5,8]])

In [90]:
num_classes = 3
num_train = 2
y = y.reshape(num_train,)

In [78]:
loss, grad = softmax_loss_naive(w, X, y, 0)
print(loss)
print(grad)

[ 8.72009496]
[[ 0.   0.5 -0.5]
 [-0.5  1.  -0.5]
 [-1.   1.5 -0.5]
 [ 0.   0.   0. ]]


In [80]:
scores = np.dot(X,w)
scores

array([[23, 37, 38],
       [32, 53, 51]])

In [100]:
exp_sum = np.sum(np.exp(scores),axis = 1).reshape(num_train,1)
exp_sum

array([[  4.35750839e+16],
       [  1.18231085e+23]])

In [108]:
correct_index = np.eye(num_classes)[y].astype('int')
correct = np.exp(scores[np.where(correct_index)].reshape(num_train,1))
print(correct_index)
correct

[[1 0 0]
 [0 0 1]]


array([[  9.74480345e+09],
       [  1.40934908e+22]])

In [110]:
loss = np.mean(-np.log(correct/exp_sum))
loss

8.7200949614307781

In [102]:
dW = np.dot(X.T,scores/exp_sum)
dW

array([[  5.27825113e-16,   8.49109991e-16,   8.72058851e-16],
       [  1.05564996e-15,   1.69821953e-15,   1.74411727e-15],
       [  1.58347480e-15,   2.54732908e-15,   2.61617569e-15],
       [  5.27824843e-16,   8.49109543e-16,   8.72058420e-16]])

In [103]:
dW = dW/num_train
dW

array([[  2.63912557e-16,   4.24554995e-16,   4.36029426e-16],
       [  5.27824978e-16,   8.49109767e-16,   8.72058636e-16],
       [  7.91737399e-16,   1.27366454e-15,   1.30808785e-15],
       [  2.63912421e-16,   4.24554771e-16,   4.36029210e-16]])

In [ ]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

In [ ]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 5e-7]
regularization_strengths = [2.5e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
pass
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

In [ ]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

In [ ]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
    
    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])

## test

In [118]:
from keras.layers import *
from keras.models import *
from keras.applications import *
from keras.optimizers import *
from keras.regularizers import *
from keras.applications.inception_v3 import preprocess_input

In [119]:
width = 299

def get_features(MODEL, data=X, prefix=''):
    cnn_model = MODEL(include_top=False, input_shape=(width, width, 3), weights='imagenet')
    
    inputs = Input((width, width, 3))
    x = inputs
    x = Lambda(preprocess_input, name='preprocessing')(x)
    x = cnn_model(x)
    x = GlobalAveragePooling2D()(x)
    cnn_model = Model(inputs, x)

    features = cnn_model.predict(data, batch_size=64, verbose=1)
    
    with h5py.File("./cache/{0}fea_{1}.h5".format(prefix, MODEL.__name__)) as h:
        h.create_dataset("features", data=features)
        
    return features

In [122]:
inception_features = get_features(InceptionV3, X, 'train_')
xception_features = get_features(Xception, X, 'train_')
inceRes_features = get_features(InceptionResNetV2, X, 'train_')
resnet_features = get_features(ResNet50, X, 'train_')

features = np.concatenate([inception_features, xception_features, inceRes_features], axis=-1)
features.shape

ValueError: Error when checking : expected input_2 to have 4 dimensions, but got array with shape (2, 4)